## Core Diversity Analyses of the GCMP dataset

This notebook will perform basic QIIME2 analysis of the GCMP dataset. The goal is not to produce interesting analytical products at this time, but rather to calculate basic diversity parameters that can be used in downstream phylogenetic comparisons. 

In [144]:
from qiime2 import Artifact
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.taxa.methods import filter_table,collapse

#The below try/except block is unsightly but the alpha function got moved between recent versions of QIIME2
#and it's nice if the notebook is compatible with either
try:
    from qiime2.plugins.diversity.methods import alpha
    from qiime2.plugins.diversity.methods import alpha_phylogenetic
except:
    from qiime2.plugins.diversity.pipelines import alpha
    from qiime2.plugins.diversity.pipelines import alpha_phylogenetic
    
from qiime2.plugins.diversity.visualizers import alpha_group_significance

from qiime2.plugins.feature_table.methods import rarefy
from qiime2.plugins.feature_table.visualizers import summarize

from qiime2.metadata import Metadata
from os.path import abspath,exists,join
from os import mkdir
import shutil

import pandas as pd

#### Check QIIME2 version
Do a quick check that the qiime version is what's expected. If you get an error at this step due to a different qiime2 verison, the code may very well still work, but if you want to exactly reproduce the results, you'll want QIIME2 2020.8.0

In [79]:
from qiime2 import __version__ as qiime_version

if qiime_version != "2020.8.0":
    raise ValueError("This code was developed with QIIME2 2020.8.0. It will *probably* work with related versions, but there are no guarantees as some functions may change in call signature.")

#### Set up variables for required files

In [80]:
input_directory = abspath("../input")
mapping_file = join(input_directory,"GCMP_EMP_map_r28_no_empty_samples.txt")
taxonomy_file = join(input_directory,"silva_metaxa2_reference_taxonomy.qza")
feature_table_file = join(input_directory,"physeq_feature_table.qza")
tree_file = join(input_directory,"physeq_rooted_tree.qza")

output_dir = abspath("../output/")

ft_output_dir = join(output_dir,"feature_tables")

if not exists(ft_output_dir):
    print(f"Output directory {ft_output_dir} does not yet exist, creating it...")
    mkdir(ft_output_dir)
    print("Done.")

required_files = [feature_table_file,mapping_file,taxonomy_file,tree_file]



Output directory /Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables does not yet exist, creating it...
Done.


#### Check that all required files really exist and are named correctly

In [81]:
print("Verifying that all needed starting data files exist.")
for existing_file in required_files:
    if not exists(existing_file):
        raise IOError(f"Required file {existing_file} not found. Please ensure it is in that directory.")
print("Done.")

Verifying that all needed starting data files exist.
Done.


#### Load input files as QIIME2 Artifacts or Metadata objects

In [82]:
from qiime2.plugins.feature_table.methods import filter_features
feature_table = Artifact.load(feature_table_file)
tree = Artifact.load(tree_file)
taxonomy = Artifact.load(taxonomy_file)
metadata = Metadata.load(mapping_file)



#### Document the starting state of the feature table out of phyloseq with a summary

In [83]:
summary_visualization = summarize(feature_table,sample_metadata=metadata)
vis = summary_visualization.visualization
output_filename = f"phyloseq_feature_table_summary.qzv"
output_filepath = join(ft_output_dir,output_filename)
print(f"Saving summary file to:{output_filepath}")
vis.save(output_filepath)

/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


Saving summary file to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/phyloseq_feature_table_summary.qzv


'/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/phyloseq_feature_table_summary.qzv'

#### Split the feature table into mucus, tissue, and skeleton tables

In addition to coral mucus, tissue and skeleton, the GCMP includes experimental blanks, water, sediment, and outgroup samples. Here's a full summary:
`
Coral Mucus
Coral Skeleton
Coral Tissue
Whole Coral
Heliopora Mucus
Heliopora Skeleton
Heliopora Tissue
Alcyonacea Mucus
Alcyonacea Skeleton
Millepora Mucus
Millepora Skeleton
Millepora Tissue
Zooanthid Mucus
Zooanthid Skeleton
Zooanthid Tissue
Reef Sediment
Reef Water
Aiptasia Tissue
Aurelia Tissue
Stylaster Tissue
Entacmaea Mucus
Macrorhynchia mucus
Macrorhynchia tissue
Ctenophore Tissue
Palythoa Mucus
Palythoa Tissue
Rhodactis
Sinularia Mucus
Sinularia Tissue
Xenia Tissue
Unknown
Missing: Not collected
`

In the next step we'll extract coral mucus, tissue, and skeleton compartments from the feature table using the tissue compartment column. This abbreviates mucus as M, tissue as T and skeleton as S. It will include outgroups into these categories if they had mucus, tissue or skeleton samples respectively.

In [84]:
#split into mucus, tissue, and skeleton compartments
feature_tables = {"all":feature_table}
compartments = ['M', 'T', 'S']
compartment_names = {"M":"mucus","S":"skeleton","T":"tissue"}
for compartment in compartments:
    compartment_name = compartment_names[compartment]
    print(f"Separating samples for compartment {compartment}")
    where = f"tissue_compartment = '{compartment}'"
    filtered_table, = filter_samples(feature_table, metadata = metadata,where = where)
    feature_tables[compartment_name]=filtered_table
    
print("Done filtering")
print(f"Resulting feature tables:",feature_tables)
    
    


Separating samples for compartment M
Separating samples for compartment T
Separating samples for compartment S
Done filtering
Resulting feature tables: {'all': <artifact: FeatureTable[Frequency] uuid: 7b24d003-80c4-44e5-9c5d-cfff55334e99>, 'mucus': <artifact: FeatureTable[Frequency] uuid: 29ed9cc9-af18-4d3d-ade1-e2f645088c04>, 'tissue': <artifact: FeatureTable[Frequency] uuid: d9a0a68a-ec9f-4cf5-898a-f4c6f932e7d7>, 'skeleton': <artifact: FeatureTable[Frequency] uuid: 3ceaf8e3-003b-4a84-a5c2-bb159779f226>}


#### Filter the feature tables to exclude mitochondria and chloroplasts

Note that it is critical to use the taxonomies supplemented with MeTaxa2 chloroplasts (from the organelle removal step) to avoid high numbers of misannotated mitochondria (typically showing up as 'Unclassified bacteria')

In [85]:
filtered_feature_tables = {}
for compartment,table in feature_tables.items():
    print("Removing mitochondria from:", compartment,table)
    #NOTE: the QIIME2 api does NOT return a single object (as I thought based on the  documentation, but a NamedTuple
    #structure with each output in it)
    filter_table_results = filter_table(table,taxonomy,exclude="mitochondria,chloroplast",mode="contains")
    filtered_table = filter_table_results.filtered_table
    filtered_feature_tables[compartment]=filtered_table
    
    #Save the resulting feature table to disk
    output_filename = f"feature_table_{compartment}.qza"
    output_filepath = join(ft_output_dir,output_filename)
    print(f"Saving results to:{output_filepath}")
    filtered_table.save(output_filepath)

print("Filtered feature tables:",filtered_feature_tables)


    
    

Removing mitochondria from: all <artifact: FeatureTable[Frequency] uuid: 7b24d003-80c4-44e5-9c5d-cfff55334e99>
Saving results to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_all.qza
Removing mitochondria from: mucus <artifact: FeatureTable[Frequency] uuid: 29ed9cc9-af18-4d3d-ade1-e2f645088c04>
Saving results to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_mucus.qza
Removing mitochondria from: tissue <artifact: FeatureTable[Frequency] uuid: d9a0a68a-ec9f-4cf5-898a-f4c6f932e7d7>
Saving results to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_tissue.qza
Removing mitochondria from: skeleton <artifact: FeatureTable[Frequency] uuid: 3ceaf8e3-003b-4a84-a5c2-

#### Output Summaries of the filtered feature tables

Note that this will produce some warnings (regarding headers in the .csv files), but these are safe to ignore. 

In [86]:
for compartment,table in filtered_feature_tables.items():
    #Output a sample summary
    summary_visualization = summarize(table,sample_metadata=metadata)
    vis = summary_visualization.visualization
    output_filename = f"feature_table_{compartment}_summary.qzv"
    output_filepath = join(ft_output_dir,output_filename)
    print(f"Saving summary file to:{output_filepath}")
    vis.save(output_filepath)

/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


Saving summary file to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_all_summary.qzv


/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


Saving summary file to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_mucus_summary.qzv


/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


Saving summary file to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_tissue_summary.qzv


/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/Users/jzaneveld/anaconda3/envs/qiime2-2020.8/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


Saving summary file to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_skeleton_summary.qzv


#### Rarefy the filtered feature tables to equal depth

Removal of mitochondria and chloroplasts may affect sequencing depth in each compartment.

In [87]:
rarefaction_depth = 1000
rarefied_feature_tables = {}
for compartment,table in filtered_feature_tables.items():
    print(f"Rarefying: {compartment} feature table {table} to {rarefaction_depth} sequences/sample")
    rarefy_results = rarefy(table=table, sampling_depth=rarefaction_depth)
    #Get the rarefied table out of the NamedTuple of results
    rarefied_filtered_table = rarefy_results.rarefied_table
    
    
    #Save the resulting feature table to disk
    output_filename = f"feature_table_{compartment}_{rarefaction_depth}.qza"
    output_filepath = join(ft_output_dir,output_filename)
    print(f"Saving results to:{output_filepath}")
    rarefied_filtered_table.save(output_filepath)
    
    #Store rarefied feature table in a dict so we don't have to reload
    rarefied_feature_tables[compartment]=rarefied_filtered_table
    

Rarefying: all feature table <artifact: FeatureTable[Frequency] uuid: 4099600e-31e1-4b05-95b1-defac3e9084f> to 1000 sequences/sample
Saving results to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_all_1000.qza
Rarefying: mucus feature table <artifact: FeatureTable[Frequency] uuid: e88ced54-bb81-4c4c-bfc9-f4b5e9b15e76> to 1000 sequences/sample
Saving results to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_mucus_1000.qza
Rarefying: tissue feature table <artifact: FeatureTable[Frequency] uuid: 1d1719ae-34db-4cd8-8fe8-aed35453d46b> to 1000 sequences/sample
Saving results to:/Users/jzaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/output/feature_tables/feature_table_tissue_1000.qza
Rarefying: skeleton f

## Species - specific analysis for trait table

Since we are comparing species for many analyses, we must use phylogenetically-aware methods. This implies that we must summarize data in terms of the tip names in our reference phylogeny. 

**Strategy:** the tip names are listed in the Huang_Roy_tree_name column of the metadata sheet. We will look up these tip names, find the unique set of names, then iteratively calculate alpha diversity for each. 


#### Find Unique Huang Roy tree names


In [99]:
# Find unique Huang Roy tree names
df = metadata.to_dataframe()
unique_species_names = list(set(list(df['Huang_Roy_tree_name'])))
print(f"There are {len(unique_species_names)} unique species names in the dataset:")
print(sorted(unique_species_names))

There are 88 unique species names in the dataset:
['ACR_Acropora', 'ACR_Acropora_cytherea', 'ACR_Acropora_humilis', 'ACR_Acropora_hyacinthus', 'ACR_Isopora_palifera', 'ACR_Montipora', 'ACR_Montipora_aequituberculata', 'ACR_Montipora_hispida', 'AGA_Gardineroseris_planulata', 'AGA_Pachyseris', 'AGA_Pavona', 'AGA_Pavona_clavus', 'AGA_Pavona_frondifera', 'AGA_Pavona_varians', 'AST_Stephanocoenia_intersepta', 'DEN', 'DEN_Dendrophyllia', 'DEN_Tubastraea', 'DEN_Turbinaria', 'EUP_Physogyra_lichtensteini', 'EUP_Plerogyra', 'FAV', 'FAV_Caulastraea_furcata', 'FAV_Cyphastrea', 'FAV_Cyphastrea_micropthalma', 'FAV_Cyphastrea_serailia', 'FAV_Diploastrea_heliopora', 'FAV_Diploria_labyrinthiformis', 'FAV_Echinopora_horrida', 'FAV_Echinopora_lamellosa', 'FAV_Echinopora_mammiformis', 'FAV_Favia_pallida', 'FAV_Favites', 'FAV_Favites_abdita', 'FAV_Goniastrea', 'FAV_Goniastrea_edwardsi', 'FAV_Goniastrea_favulus', 'FAV_Leptastrea_transversa', 'FAV_Leptoria_phrygia', 'FAV_Montastraea_annularis', 'FAV_Montastr

In [188]:
from IPython.core.display import HTML
def calculate_per_species_diversities(feature_table,\
                                      metadata,\
                                      species_column = "Huang_Roy_tree_name",\
                                      compartment_name = 'all',\
                                      taxonomy = None,\
                                      metrics = ['faith_pd','observed_features','gini_index','dominance','simpson_e'],\
                                      to_skip = ['none','','Not applicable','Missing: Not collected']\
                                      ):
    #Set up a DataFrame to hold results
    results_columns = [species_column,f"n_samples_{compartment_name}"]
    results_columns.extend([f"{metric}_{compartment_name}" for metric in metrics])
    taxonomy_levels = ('domain','phylum','class','order','family','genus')
    taxonomy_labels = [f"most_abundant_{level}_{compartment_name}" for level in taxonomy_levels]
    results_columns.extend(taxonomy_labels)
    print("Result columns:",results_columns)
    results_df = pd.DataFrame(columns = results_columns)
    results_df = results_df.set_index(species_column)
    metadata_df = metadata.to_dataframe()
    unique_species_names = list(set(list(metadata_df[species_column])))
    for species in unique_species_names:
        if species in to_skip:
            continue
            
        #Filter the feature table to just our current species
        where = f"[{species_column}] = '{species}'"
        filter_results = filter_samples(feature_table, metadata = metadata,where = where)
        species_table = filter_results.filtered_table
        #We'll use the species table, not the overall feature table from here on down!
        
        
        #If taxonomy is provided, summarize the type of microbe with highest average
        #abundance at each taxonomic level
        if taxonomy:
            for i, taxon_label in enumerate(taxonomy_levels):
                level = i + 1 #domain is level 1 in QIIME2, not level 0
                most_abundant_taxon = get_dominant_taxon(species_table,taxonomy,level=level)
                column_label = f"most_abundant_{taxon_label}_{compartment_name}"
                results_df.loc[species,column_label] = most_abundant_taxon
            
        
        for metric in metrics:
            #print(f"Calculating {metric} for {species}")
            try:
                if metric == "faith_pd":
                    alpha_results = alpha_phylogenetic(species_table,phylogeny=tree,metric=metric)
                else:
                    alpha_results = alpha(species_table, metric = metric)
            except ValueError:
                print(f"Can't calculate {metric} for {species} {compartment_name}...skipping")
                continue
            
            alpha_diversity = alpha_results.alpha_diversity
            species_adiv = alpha_diversity.view(pd.Series)
            
            species_mean = species_adiv.mean()
            results_df.loc[species,f"{metric}_{compartment_name}"] = species_mean
            print(f"{species}\t{metric}\t{compartment_name}\t{round(species_mean,4)}")
            
            
            
        #For last metric only we'll grab n (should be the same for all)
        species_n = len(species_adiv)     
        results_df.loc[species,f"n_samples_{compartment_name}"]=species_n
    return results_df
 
#calculate the dominant taxon family
def get_dominant_taxon(feature_table,taxonomy,level=5):
    """Collapse the feature table at the specified level, then find which feature is most abundant"""
    try:
        collapse_results = collapse(feature_table,taxonomy,level)
        taxon_table = collapse_results.collapsed_table
        taxon_df = taxon_table.view(pd.DataFrame)
        #Calculate average abundance of each taxon in this species
    except TypeError:
        return None
    taxon_abundance_dict = {taxon_df[col].mean():col for col in list(taxon_df.columns)}
    greatest_mean_abundance = sorted(taxon_abundance_dict.keys())[-1]
    most_abundant_taxon = taxon_abundance_dict[greatest_mean_abundance]
    print("Most abundant taxon:",most_abundant_taxon)
    return most_abundant_taxon
    
    
results_dfs = {} 
for compartment,table in sorted(rarefied_feature_tables.items(),reverse=True):
    result_df = calculate_per_species_diversities(table,metadata,compartment_name = compartment,taxonomy = taxonomy)
    species_adiv_path = join(output_dir,f"adiv_trait_table_{compartment}.tsv")
    results_dfs[compartment]=result_df
    result_df.to_csv(species_adiv_path,sep="\t")

Result columns: ['Huang_Roy_tree_name', 'n_samples_tissue', 'faith_pd_tissue', 'observed_features_tissue', 'gini_index_tissue', 'dominance_tissue', 'simpson_e_tissue', 'most_abundant_domain_tissue', 'most_abundant_phylum_tissue', 'most_abundant_class_tissue', 'most_abundant_order_tissue', 'most_abundant_family_tissue', 'most_abundant_genus_tissue']
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales;D_4__Burkholderiaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales;D_4__Burkholderiaceae;D_5__Aquabacterium
AGA_Pachyseris	faith_pd	tissue	42.604
AGA_Pachyseris	observed_features	tissue	91.125
AGA_Pachys

AST_Stephanocoenia_intersepta	faith_pd	tissue	68.8823
AST_Stephanocoenia_intersepta	observed_features	tissue	138.2
AST_Stephanocoenia_intersepta	gini_index	tissue	0.9386
AST_Stephanocoenia_intersepta	dominance	tissue	0.4568
AST_Stephanocoenia_intersepta	simpson_e	tissue	0.0254
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Pseudomonadales
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Pseudomonadales;D_4__Pseudomonadaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Pseudomonadales;D_4__Pseudomonadaceae;D_5__Pseudomonas
AGA_Pavona	faith_pd	tissue	91.2272
AGA_Pavona	observed_features	tissue	237.0
AGA_Pavona	gini_index	tissue	0.9419
AGA_Pavona	dominance	tissue	0.0664
AGA_Pavona	simpson_e	tissue	0.1628
Most 

EUP_Physogyra_lichtensteini	faith_pd	tissue	27.1121
EUP_Physogyra_lichtensteini	observed_features	tissue	53.4
EUP_Physogyra_lichtensteini	gini_index	tissue	0.9306
EUP_Physogyra_lichtensteini	dominance	tissue	0.1307
EUP_Physogyra_lichtensteini	simpson_e	tissue	0.1744
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales;D_4__Stappiaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales;D_4__Stappiaceae;__
DEN_Turbinaria	faith_pd	tissue	83.7741
DEN_Turbinaria	observed_features	tissue	214.9231
DEN_Turbinaria	gini_index	tissue	0.9591
DEN_Turbinaria	dominance	tissue	0.1869
DEN_Turbinaria	simpson_e	tissue	0.2765
Most abundant taxon: D_0__Bac

Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhodobacterales;D_4__Rhodobacteraceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhodobacterales;D_4__Rhodobacteraceae;__
DEN_Dendrophyllia	faith_pd	tissue	84.6634
DEN_Dendrophyllia	observed_features	tissue	205.0
DEN_Dendrophyllia	gini_index	tissue	0.7954
DEN_Dendrophyllia	dominance	tissue	0.0416
DEN_Dendrophyllia	simpson_e	tissue	0.1179
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae;D_5_

Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Sphingomonadales;D_4__Sphingomonadaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Sphingomonadales;D_4__Sphingomonadaceae;D_5__Sphingobium
ACR_Acropora_humilis	faith_pd	tissue	69.7876
ACR_Acropora_humilis	observed_features	tissue	205.0
ACR_Acropora_humilis	gini_index	tissue	0.5694
ACR_Acropora_humilis	dominance	tissue	0.0163
ACR_Acropora_humilis	simpson_e	tissue	0.2989
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales;D_4__Rhizobiaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales;D_4__Rhizobiaceae;__


...

Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Vibrionales
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Vibrionales;D_4__Vibrionaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Vibrionales;D_4__Vibrionaceae;D_5__Vibrio
FAV_Platygyra_daedalea	faith_pd	all	55.4022
FAV_Platygyra_daedalea	observed_features	all	116.5
FAV_Platygyra_daedalea	gini_index	all	0.9463
FAV_Platygyra_daedalea	dominance	all	0.1029
FAV_Platygyra_daedalea	simpson_e	all	0.1115
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;__
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;__;__
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;__;__;__
SID_P

Most abundant taxon: D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Bacteroidales;D_4__Rikenellaceae
Most abundant taxon: D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Bacteroidales;D_4__Rikenellaceae;D_5__Rikenellaceae RC9 gut group
FUN_Ctenactis_echinata	faith_pd	all	44.4168
FUN_Ctenactis_echinata	observed_features	all	78.0
FUN_Ctenactis_echinata	gini_index	all	0.8787
FUN_Ctenactis_echinata	dominance	all	0.324
FUN_Ctenactis_echinata	simpson_e	all	0.1278
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonad

Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae;D_5__Endozoicomonas
MER_Hydnophora_exesa	faith_pd	all	29.4846
MER_Hydnophora_exesa	observed_features	all	57.7273
MER_Hydnophora_exesa	gini_index	all	0.9702
MER_Hydnophora_exesa	dominance	all	0.3693
MER_Hydnophora_exesa	simpson_e	all	0.1214
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Sphingomonadales
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Sphingomonadales;D_4__Sphingomonadaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Sphingomonadales;D_4__

FAV_Diploria_labyrinthiformis	simpson_e	all	0.1989
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales;__
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales;__;__
PEC_Echinophyllia_orpheensis	faith_pd	all	98.4726
PEC_Echinophyllia_orpheensis	observed_features	all	261.6667
PEC_Echinophyllia_orpheensis	gini_index	all	0.8728
PEC_Echinophyllia_orpheensis	dominance	all	0.1451
PEC_Echinophyllia_orpheensis	simpson_e	all	0.0616
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
Most abundant taxo

Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales;D_4__Stappiaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales;D_4__Stappiaceae;D_5__Pseudovibrio
PEC_Echinophyllia_aspera	faith_pd	all	57.644
PEC_Echinophyllia_aspera	observed_features	all	136.1
PEC_Echinophyllia_aspera	gini_index	all	0.9615
PEC_Echinophyllia_aspera	dominance	all	0.2558
PEC_Echinophyllia_aspera	simpson_e	all	0.1605
Most abundant taxon: D_0__Bacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
Most abundant taxon: D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Flavobacteriales
Most abundant taxon: D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Flavobacteriales;D_4__Flavobacteriaceae
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Pseudomonadales;D_4__Pseudomonadaceae;D_5__Pseudomona

#### Integrate microbiome data into per-species trait table

In order to allow microbiome data to be integrated into evolutionary analysis, we now join our per-host-species microbiome statistics with the trait table describing disease prevalence in each species.

In [189]:
from IPython.core.display import HTML,display

disease_df = pd.read_csv("../input/merged_disease_table.tsv",sep="\t")
disease_df = disease_df.set_index('Species')
for compartment,results_df in results_dfs.items():
    results_df.index.rename('Species')    
    disease_df = pd.merge(disease_df, results_df, how="outer", left_index = True, right_index = True, indicator=False)
disease_df.to_csv("../output/GCMP_trait_table.tsv",sep="\t")


In [190]:
display(HTML(disease_df.to_html()))

,Unknown_disease_FRRP,Black_band_disease_FRRP,Dark_spots_FRRP,Red_band_FRRP,White_band_FRRP,White_plague_FRRP,White_spot_FRRP,Yellow_band_FRRP,Algal_infection_HICORDIS,Algal_overgrowth_HICORDIS,Black_band_disease_HICORDIS,Bleaching_HICORDIS,Ciliates_HICORDIS,Corallophila_huysmansii_HICORDIS,Cyanobacteria_HICORDIS,Discoloration_HICORDIS,Endolithic_fungal_infection_HICORDIS,Endolithic_hypermycosis_HICORDIS,Gear_entanglement_HICORDIS,Growth_anomalies_HICORDIS,Macroalgal_overgrowth_HICORDIS,No_Disease_HICORDIS,Pigmentation_response_HICORDIS,Predation_HICORDIS,Recently_denuded_skeleton_HICORDIS,Swollen_patches_HICORDIS,Tissue_loss_HICORDIS,Trematodiasis_HICORDIS,White_syndrome_HICORDIS,Total_diseased_Lamb_et_al,Total_compromised_or_bleached_Lamb_et_al,White_syndrome_Lamb_et_al,Black_band_disease_Lamb_et_al,Brown_band_Lamb_et_al,Skeletal_eroding_band_Lamb_et_al,Growth_anomalies_Lamb_et_al,n_samples_tissue,faith_pd_tissue,observed_features_tissue,gini_index_tissue,dominance_tissue,simpson_e_tissue,most_abundant_domain_tissue,most_abundant_phylum_tissue,most_abundant_class_tissue,most_abundant_order_tissue,most_abundant_family_tissue,most_abundant_genus_tissue,n_samples_skeleton,faith_pd_skeleton,observed_features_skeleton,gini_index_skeleton,dominance_skeleton,simpson_e_skeleton,most_abundant_domain_skeleton,most_abundant_phylum_skeleton,most_abundant_class_skeleton,most_abundant_order_skeleton,most_abundant_family_skeleton,most_abundant_genus_skeleton,n_samples_mucus,faith_pd_mucus,observed_features_mucus,gini_index_mucus,dominance_mucus,simpson_e_mucus,most_abundant_domain_mucus,most_abundant_phylum_mucus,most_abundant_class_mucus,most_abundant_order_mucus,most_abundant_family_mucus,most_abundant_genus_mucus,n_samples_all,faith_pd_all,observed_features_all,gini_index_all,dominance_all,simpson_e_all,most_abundant_domain_all,most_abundant_phylum_all,most_abundant_class_all,most_abundant_order_all,most_abundant_family_all,most_abundant_genus_all
ACR_Acropora,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.913043,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.032284,0.108322,0.004980,0.006148,0.005461,0.014493,0.001030,5,24.8399,44.2,0.919456,0.141614,0.190916,D_0__Bacteria,D_0__Bacteria;D_1__Proteobacteria,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales;D_4__Burkholderiaceae,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae;D_5__Endozoicomonas,3,31.3213,60.3333,0.894675,0.136019,0.173142,D_0__Bacteria,D_0__Bacteria;D_1__Proteobacteria,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae;D_5__Endozoicomonas,3,19.4749,34.3333,0.888689,0.249417,0.190328,D_0__Bacteria,D_0__Bacteria;D_1__Proteobacteria,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales;D_4__Endozoicomonadaceae;D_5__Endozoicomonas,11,24.7917,46.0909,0.956624,0.171814,0.187559,D_0__Bacteria,D_0__Bacteria;D_1__Proteobacteria,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospirillales,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Oceanospiri